# Script to generate urban layouts from the citygenerator package
Generates urban layouts based on the input parameters building density $\lambda_p$ and frontal aspect ratio $\lambda_f$. Randomisation by chosing a fractal type for the street layout ("hierarchical", "cascade" or "random") and a randomness parameter $r$.

# Contents
[Setup](#Setup)

[Generate layout](#Generate-layout)

[Video of generation process](#Video-of-generation-process)

[Write data](#Write-data)

[Run checks](#Run-checks-for-generated-layout)

---
# Setup

#### Set up libaries and paths

In [ ]:
%matplotlib inline
import sys, os
import math
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('citygenerator')
import citygenerator as cgen

#### Set up domain and resolution

In [ ]:
imax = 120  # cells in x
jtot = 120  # cells in y
kmax = 180  # cells in z
xsize = 240  # domain size in x (in m)
ysize = 240  # domain size in y (in m)
zsize = 180  # domain size in z (in m)

#### Generate the grids

In [ ]:
# Processing x-grid
dx = xsize/imax
xedge = np.arange(0, xsize, dx)
xcentre = xedge + dx*0.5
xgrid = xcentre

# Processing y-grid
dy = ysize/jtot
yedge = np.arange(0, ysize, dy)
ycentre = yedge + dy*0.5
ygrid = ycentre

# Processing z-grid    
dz = zsize/kmax
zedge = np.arange(0, zsize, dz)
zcentre = zedge + dz*0.5
dzs = [dz] * kmax

---
# Generate layout
[Back to contents](#Contents)

#### Set layout parameters

In [ ]:
lp = 0.45  # Building density (0 to 1)
lg = 0.1  # Greenery density (0 to 1)
lf = 0.19  # Frontal area (0 to 1)
ldegree = 0.6  # degree of randomness for layout (0 to 1)
hdegree = 0.4  # degree of randomness for height (0 to 1)
o = 'r'  # "r for 'random', h for 'hierarchical' and c for 'cascade'

## Generate a single block with parameters

In [ ]:
# blocks = cgen.oneblock.generate(xsize, ysize, lp, lf, margin=8, exact=False)
# greenery = []
# lpfinal = cgen.utils.blockplan(blocks)/(xsize*ysize)
# lffinal = cgen.utils.blockfront(blocks)/(xsize*ysize)
# print("Blocks:", blocks)
# print("lp = ", round(lpfinal, 3))
# print("lf = ", round(lffinal, 3))

## Generate fractal blocks

In [ ]:
savesteps=True  # whether to save step by step creation
blocks, greenery, blockgeneration = cgen.fractal.generate(xsize, ysize, zsize, imax, jtot, kmax,
                                                          lp, lg, lf, 
                                                          order=o,
                                                          layoutrandom=ldegree, heightrandom=hdegree, 
                                                          margin=5, minwidth=8, minvolume=10,
                                                          savesteps=savesteps)
lpfinal = cgen.utils.blockplan(blocks)/(xsize*ysize)
lffinal = cgen.utils.blockfront(blocks)/(xsize*ysize)
# print("Blocks:", blocks)
print("lp = ", round(lpfinal, 2))
print("lf = ", round(lffinal, 2))

## Display block layout

In [ ]:
limits=[0, xsize, 0, ysize, 0, zsize/2]
fig = plt.figure(figsize=(3*2, 4), dpi=150)
# geometry plot
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
cgen.plot.blocks3d(blocks, ax=ax1, limits=limits, edgecolor='k')
cgen.plot.blocks3d(greenery, ax=ax1, limits=limits, facecolor='g')
# layout plot
ax2 = fig.add_subplot(1, 2, 2)
cgen.plot.layout(blocks, ax=ax2, limits=limits[:4])
cgen.plot.layout(greenery, ax=ax2, limits=limits[:4], facecolor='g')
plt.tight_layout()
plt.show()

## Block statistics

In [ ]:
blockstats = cgen.utils.blockstats(blocks, a0=xsize*ysize)
print("lamda_p = ", blockstats['lp'])
print("lambda_f = ", blockstats['lf'])
print("zmax = ", blockstats['zmax'])
print("zh = ", blockstats['zh'])
print("no. of blocks: ", blockstats['nblocks'])
print("blocks:\n", blocks)

---
# Video of generation process
[Back to contents](#Contents)

In [ ]:
saveimage=False
showvideo=True
limits = [0, xsize, 0, ysize, 0, zsize/2]
if blockgeneration:  # if list of steps not empty
    if saveimage is True:
        savehere = cgen.plot.save_path("videos")
#         save_opts = {'transparent' : True, 'bbox_inches' : 'tight'}
        save_opts = {'transparent' : False}
    fig = plt.figure(figsize=(16*0.5, 9*0.5), dpi=120)
    fig.suptitle("Urban landscape generator")
    ax = fig.add_subplot(1, 1, 1)
    for i, itblocks in enumerate(blockgeneration):
        if len(itblocks[0]) == 4:  # a block in itblocks is 2D
            ax.clear()  # clear data from axis
            ax.set_title("Step %s" % i)
            cgen.plot.layout(itblocks, ax=ax, limits=limits[:4])
            fig.subplots_adjust(top=0.85, bottom=0.1)
            if saveimage is True:
                cgen.plot.save_image(savehere + "ULG", i, **save_opts)
            if showvideo is True:
                cgen.plot.show_video(fig)
        elif len(itblocks[0]) == 6:  # a block in itblocks is 3D
            plt.clf()  # clear plot data
            fig.suptitle("Urban landscape generator")
            ax = fig.add_subplot(1, 1, 1, projection='3d')
            cgen.plot.blocks3d(itblocks, ax=ax, edgecolor='k', limits=limits)
            cgen.plot.blocks3d(greenery, ax=ax, facecolor='g', limits=limits)
            ax.set_title("Step %s" % i)
            ax.dist=10
            fig.subplots_adjust(top=0.915)
            if saveimage is True:
                cgen.plot.save_image(savehere + "ULG", i, **save_opts)  
            if showvideo is True:
                cgen.plot.show_video(fig)
    plt.show()

---
# Write data
[Back to contents](#Contents)

## Change resolution or convert blocks to grid indices

In [ ]:
dx = imax/xsize
dy = jtot/ysize
dz = kmax/zsize
block_indices = cgen.utils.convert(blocks, dx, dy, dz, rounding=True)
print(block_indices)

## Write the data

In [ ]:
savedir = "./layouts"
if not os.path.exists(savedir):
    os.mkdir(savedir)
blocksfile = os.path.join(savedir, 'blocks.txt')
cgen.utils.write(block_indices, blocksfile)

---
# Run checks for generated layout
[Back to contents](#Contents)

#### Check block to height ratio

In [ ]:
problemblocks = cgen.checks.check_heightratio(blocks, zsize, heightratio=6)

#### Check volume per block

In [ ]:
dx = xsize/imax
dy = ysize/jtot
dz = zsize/kmax
dimfactor = dx*dy*dz
problemblocks = cgen.checks.check_blockvolume(blocks, blockvolume=6*dimfactor)

#### Check canyon width
This function might not always work, canyons are still in development

In [ ]:
# dx = xsize/imax
# problemcanyons = cgen.checks.check_canyonwidths(blocks, [0, xsize, 0, ysize], canyonwidth=5*dx)

#### Run all checks and plot problematic blocks and canyons

In [ ]:
domain=[0, xsize, 0, ysize, 0, zsize]
dx = xsize/imax
dy = ysize/jtot
dz = zsize/kmax
resolution = [dx, dy, dz]
problemblocks, problemcanyons = cgen.checks.allchecks(blocks, domain, resolution,
                                                    heightratio=6, 
                                                    blockvolume=10, 
                                                    canyonwidth=4*dx)
if problemblocks or problemcanyons:  # if list is not empty
    fig = plt.figure(figsize=(3*2, 4), dpi=100)
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    cgen.plot.blocks3d(problemblocks, ax=ax1, edgecolor='k', 
                       limits=domain)
    ax2 = fig.add_subplot(1, 2, 2)
    cgen.plot.layout(problemblocks, ax=ax2, limits=domain[:4])
    cgen.plot.layout(problemcanyons, ax=ax2, limits=domain[:4], facecolor='grey')
    plt.suptitle("Problematic blocks and canyons")    
    plt.tight_layout()
    plt.show()

[Back to contents](#Contents)